In [7]:
from skimage.util import view_as_windows, img_as_uint, img_as_ubyte, img_as_float
from skimage import io, exposure
from glob import glob
import os
import numpy as np
import shutil
from random import shuffle

In [52]:
raw_imgs = glob('raw_data\\PSR\\*.tif') #+ glob('raw_data\\Unstained\\*.tif')
# raw_imgs = glob('E:\\experiments_drive\\experiments\\dsmil\\WSI\\mh_fovs_SHG\\New folder\\pdac\\*.tif')
outpath = 'datasettest'

In [53]:
window_shape = (512, 512)
thresh = 0.02

In [68]:
im_max = []
im_min = []
for img in raw_imgs:
    im = io.imread(img)
    im_max.append(np.percentile(im, 99.5))
    im_min.append(np.percentile(im, 23))
max_im = sum(im_max)/len(im_max)
min_im = sum(im_min)/len(im_min)

In [69]:
for img in raw_imgs:
    dataset_name = os.path.dirname(img).split(os.sep)[-1]
    im_name = os.path.splitext(os.path.basename(img))[0]
    # break
    # os.makedirs(os.path.join(outpath, im_name), exist_ok=True)
    im = io.imread(img)
    im = exposure.rescale_intensity(im, in_range=(min_im, max_im), out_range=(0, 1))
    im_windows = view_as_windows(im, window_shape, step=window_shape)
    for i in range(im_windows.shape[0]):
        for j in range(im_windows.shape[1]):
            im_tile = im_windows[i, j]
            # print(np.mean(im_tile))
            if np.mean(im_tile)>thresh:
                save_name = os.path.join(outpath, f'{dataset_name}___{im_name}___{i}_{j}.tif')
                im = im_windows[i, j]
                # im = exposure.rescale_intensity(im, in_range=(min_im, max_im), out_range=(0, 1))
                io.imsave(save_name, img_as_ubyte(im_windows[i, j]))

In [85]:
for img in raw_imgs:
    im = io.imread(img)
    im = exposure.rescale_intensity(im, in_range=(min_im, max_im), out_range=(0, 1))
    io.imsave(img, img_as_ubyte(im))

C:\Users\binli123\AppData\Local\Temp/ipykernel_484/1874270158.py:4: UserWarning: E:\experiments_drive\experiments\dsmil\WSI\mh_fovs_SHG\normal\PDAC_MetroHealth_N302L5-lsm--5968_7092.ome.tif is a low contrast image
  io.imsave(img, img_as_ubyte(im))
C:\Users\binli123\AppData\Local\Temp/ipykernel_484/1874270158.py:4: UserWarning: E:\experiments_drive\experiments\dsmil\WSI\mh_fovs_SHG\normal\PDAC_MetroHealth_N302L5-lsm--6953_5066.ome.tif is a low contrast image
  io.imsave(img, img_as_ubyte(im))
C:\Users\binli123\AppData\Local\Temp/ipykernel_484/1874270158.py:4: UserWarning: E:\experiments_drive\experiments\dsmil\WSI\mh_fovs_SHG\normal\PDAC_MetroHealth_N302L5-lsm--9026_7439.ome.tif is a low contrast image
  io.imsave(img, img_as_ubyte(im))
C:\Users\binli123\AppData\Local\Temp/ipykernel_484/1874270158.py:4: UserWarning: E:\experiments_drive\experiments\dsmil\WSI\mh_fovs_SHG\normal\PDAC_MetroHealth_N352L42-lsm--9574_6312.ome.tif is a low contrast image
  io.imsave(img, img_as_ubyte(im))
C:\

In [49]:
annotated = glob('annotations/round_1/images/*.tif')
pools = glob('datasettest/*.tif')
target_dir = 'annotations/tese/images'
os.makedirs(target_dir, exist_ok=True)

In [50]:
shuffle(pools)
annotated_imgs = [i.split(os.sep)[-1] for i in annotated]
count = 0
# img_sums = []
for img_path in pools:
    img_name = img_path.split(os.sep)[-1]
    # img_sums.append(np.sum(io.imread(img_path)))
    # if np.sum(io.imread(img_path)) < thresh: continue
    if img_name not in annotated_imgs:
        dname = img_name.split('___')[0]
        if dname == 'Cole':
            pass
        if dname == 'PCCP':
            if np.random.uniform(0, 1) > 0.5: continue
        if dname == 'Unstained':
            if np.random.uniform(0, 1) > 0.25: continue
        dst = os.path.join(target_dir, img_name)
        shutil.copyfile(img_path, dst)
        count += 1
    else:
        print('In pool')
    if count > 200: break

In [30]:
thresh = np.percentile(img_sums, 40)